# Austin, TX Crime Analysis

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from usefuls import atx_zip_codes, census_key, zipcode_tabulation_area, similar_offense_type